# 역전파 학습 알고리즘의 구현과 실험

## 설치

In [ ]:
!pip install pillow # png 파일을 불러오기 위해 사용

## import

In [30]:
import random
import math
from PIL import Image

## 파라미터

In [31]:
input_size=16*16 # 16 x 16 픽셀
output_size = 7 # t, u, v, w, x, y, z 7개 
hidden_size = 1000
learning_cycle = 100 # 학습 사이클 횟수 
data_set = 10 # 데이터 셋의 개수

## 신경망 구현  
input_layer: 16 x 16 = 256개  
hidden_layer1: 1000개  
hidden_layer2: 1000개  
output_layer: 7개  

In [32]:
input_layer = [0]*input_size # input_layer를 만들어준다 
output_layer = [0]*output_size # output_layer를 만들어준다 
hidden_layer1 = [0]*hidden_size #hidden_layer1을 만들어준다
hidden_layer2 = [0]*hidden_size #hidden_layer2를 만들어준다


w1 = [[0 for col in range(input_size)]for row in range(hidden_size)] # 첫번째 가중치 (input_layer에서 hidden_layer로 가는 가중치)
w2 = [[0 for col in range(hidden_size)]for row in range(hidden_size)] # 두번째 가중치 (hidden_layer1에서 hidden_layer2로 가는 가중치)
w3 = [[0 for col in range(hidden_size)]for row in range(output_size)] # 세번째 가중치 (hidden_layer2에서 output_layer로 가는 가중치)

## 가중치 초기화 

In [103]:
# w1 초기화
for i in range(hidden_size): # 행의 개수만큼 for문을 돌려준다 
    for j in range(input_size): # 열의 개수만큼 for문을 돌려준다 
        w1[i][j]=float(round(random.random(),4)) # i행 j열의 값을 0이상 1미만의 랜덤값으로 만들어준다 소수점 4자리까지 표시(반올림해준다)

# w2 초기화
for i in range(hidden_size): # 행의 개수만큼 for문을 돌려준다 
    for j in range(hidden_size): # 열의 개수만큼 for문을 돌려준다 
        w2[i][j]=float(round(random.random(),4)) # i행 j열의 값을 0이상 1미만의 랜덤값으로 만들어준다 소수점 4자리까지 표시(반올림해준다)

# w3 초기화
for i in range(output_size): # 행의 개수만큼 for문을 돌려준다 
    for j in range(hidden_size): # 열의 개수만큼 for문을 돌려준다 
        w3[i][j]=float(round(random.random(),4)) # i행 j열의 값을 0이상 1미만의 랜덤값으로 만들어준다 소수점 4자리까지 표시(반올림해준다)


## 행렬곱 함수

In [148]:
def matrixmult(A, B): 
    row_A = len(A)
    col_A = len(A[0])
    row_B = len(B) # B는 input 값이기 떄문에 항상 열 개수가 1이다
    
    if col_A != row_B:
        raise ValueError("행렬 곱셈이 불가능합니다")
    
    C=[0]*row_A # C의 크기는 A의 행 수 x B의 열 수 인데 B의 열 수는 1이라 제외 
    for i in range(row_A):
            for k in range(col_A):
                C[i] += A[i][k]*B[k] # 행렬곱 c[i][j] += A[i][k]*B[k][j](j는 B의 열 번호) 이지만 여기서는 B의 열 개수가 항상 1이므로 제외
    return C

## 이미지 함수

In [150]:
def img_processing(alphabet,num):
    img_path = str('./image/learning/'+(chr(alphabet+ord('t')))+str(num+1)+'.png') 
    img = Image.open(img_path) # 이미지를 불러온다
    img = img.convert("L") # 이미지를 흑백으로 바꾼다 
    img = list(img.getdata()) # 이미지 데이터를 list 형태로 바꾸어준다 
    
    return img

## 순전파 함수

In [151]:
def FP(img): 
    #### input_layer ####
    for x in range(input_size):
        input_layer[x] = float(img[x]) # input_layer에 이미지를 넣어준다
    
    #### hidden_layer1 ####
    hidden_input1 = matrixmult(w1, input_layer) # 행렬곱을 해준다 (가중치 계산 input_layer -> hidden_layer1)
            
    for x in range(hidden_size):
        hidden_layer1[x] = hidden_input1[x] # hidden_layer1에 행렬곱한 결과를 넣어준다 
            
    hidden_output1 = [0]*hidden_size # hidden_layer1의 output을 초기화 해준다 
    for x in range(hidden_size): 
        hidden_output1[x] = max(0,hidden_layer1[x]) # ReLU연산을 해준다음 hidden_output1에 저장해준다 
    
    #### hidden_layer2 ####
    hidden_input2 = matrixmult(w2, hidden_output1) # 행렬곱을 해준다 (가중치 계산 hidden_layer1 -> hidden_layer2)
    
    for x in range(hidden_size):
        hidden_layer2[x]=hidden_input2[x] # hidden_layer2에 행렬곱한 결과를 넣어준다 
        
    hidden_output2 = [0]*hidden_size # hidden_layer2의 output을 초기화 해준다 
    for x in range(hidden_size):
        hidden_output2[x]=max(0,hidden_layer2[x]) # ReLU연산을 해준다음 hidden_output2에 저장해준다 
    
    #### output_layer ####
    output_input = martixmult(w3, hidden_output2) # 행렬곱을 해준다 (가중치 계산 hidden_layer2 -> output_layer)
    
    for x in range(output_size):
        output_layer[x]=output_input[x] # output_layer에 행렬곱한 결과를 넣어준다 
        
    output_output = [0]*output_size # output_layer의 output을 초기화 해준다
    for x in range(output_size):
        output_output[x]=max(0,output_layer[x])
        
    return output_output
    
    
    

In [149]:
# 순전파 
for i in range(learning_cycle): # 학습 사이클 
    for j in range(output_size): # t~z 까지 
        for k in range(data_set): # 데이터셋의 개수만큼 반복
            # 이미지 가공 
            img = img_processing(j, k)
            output = FP(img)

            
            
            
            
                
            
            
            
            

            

103.0


TypeError: object of type 'int' has no len()